In [1]:
import pandas as pd
import numpy as np

# Import Data

We want to import all the data files as data frames, and then we want to pick the needed quantities from each to combine into one summary dataframe.

For ease, we will take one of the datasets (i.e. SACRA, Whisky, etc.) at a time, and then combine them at the end into one file

## BAM

### Simulation Parameters

In [2]:
BAMData = pd.read_csv('./Simulation_Parameters/BAMDataV2.tsv', sep='\t')
print(BAMData.head())

   Unnamed: 0 Model name     Runs Spin        M0    q      M0Ω0 EOS name  \
0           0   BAM_0001  R01-R04    0     2.699  1.0  0.037977       2B   
1           1   BAM_0002  R01-R02    0     2.699  1.0  0.037986       2H   
2           2   BAM_0003  R01-R02    0  2.700033  1.0  0.038003     ALF2   
3           3   BAM_0004  R01-R02    0  2.701928  1.0  0.051776     ALF2   
4           4   BAM_0005  R01-R02    0  2.750016  1.0  0.036025     ALF2   

          k2        EOS         e        M1        M2       Lambda   M_chirp  
0   23.76117   126.7263  0.007093  1.349976  1.349976   126.726240  1.175222  
1  436.10590  2330.0000  0.009810  1.349979  1.349979  2325.898133  1.175225  
2  137.50220   733.3450  0.008908  1.350017  1.350000   733.345067  1.175251  
3  136.89010   730.0000  0.000000  1.350964  1.350964   730.080533  1.176082  
4  123.40240   658.0000  0.006877  1.375008  1.375008   658.146133  1.197014  


In [3]:
#Define Identifier List
iden_list = []
for iden in BAMData['Model name']:
    
    iden_list.append(iden.replace('_',''))

print(iden_list)
print(len(iden_list))

['BAM0001', 'BAM0002', 'BAM0003', 'BAM0004', 'BAM0005', 'BAM0006', 'BAM0007', 'BAM0008', 'BAM0009', 'BAM0010', 'BAM0011', 'BAM0012', 'BAM0013', 'BAM0014', 'BAM0015', 'BAM0016', 'BAM0017', 'BAM0018', 'BAM0019', 'BAM0020', 'BAM0021', 'BAM0022', 'BAM0023', 'BAM0024', 'BAM0025', 'BAM0026', 'BAM0027', 'BAM0028', 'BAM0029', 'BAM0030', 'BAM0031', 'BAM0032', 'BAM0033', 'BAM0034', 'BAM0035', 'BAM0036', 'BAM0037', 'BAM0038', 'BAM0039', 'BAM0040', 'BAM0041', 'BAM0042', 'BAM0043', 'BAM0044', 'BAM0045', 'BAM0046', 'BAM0047', 'BAM0048', 'BAM0049', 'BAM0050', 'BAM0051', 'BAM0052', 'BAM0053', 'BAM0054', 'BAM0055', 'BAM0056', 'BAM0057', 'BAM0058', 'BAM0059', 'BAM0060', 'BAM0061', 'BAM0062', 'BAM0063', 'BAM0064', 'BAM0065', 'BAM0066', 'BAM0067', 'BAM0068', 'BAM0069', 'BAM0070', 'BAM0071', 'BAM0072', 'BAM0073', 'BAM0074', 'BAM0075', 'BAM0076', 'BAM0077', 'BAM0078', 'BAM0079', 'BAM0080', 'BAM0081', 'BAM0082', 'BAM0083', 'BAM0084', 'BAM0085', 'BAM0086', 'BAM0087', 'BAM0088', 'BAM0089', 'BAM0090', 'BAM0091'

#### Cell Shifted Earlier

I am going to shift the Psi4 Lev1 above because it has more terms, and so to avoid cutting them out it should go first and define iden_list2: 

Note that the iden_list2 is now going to be unused because I decided to include all of them. The reason for this is because it appears that the terms that are bad between the strain and psi4 do not perfectly match up, and so I should just include all of the runs to avoid cutting out any good ones.

In [9]:
BAM_psi4_mrg_Lev1 = pd.read_csv('./Processed/Merger_Values_BAM_Lev1.txt', sep='\t')
print(BAM_psi4_mrg_Lev1.head())

                                          File_Label  EOS  Time_Merger  \
0  R01\BAM0001_2B_1350_1350_psi4_r850_Trim_Freq_T...    1  6944.496000   
1  R01\BAM0002_2H_1350_1350_psi4_r900_Trim_Freq_T...    2  4818.780000   
2  R01\BAM0003_ALF2_1350_1350_psi4_r900_Trim_Freq...    3  5978.784000   
3  R01\BAM0004_ALF2_1351_1351_psi4_r1000_Trim_Fre...    4  2213.112000   
4  R01\BAM0005_ALF2_1375_1375_psi4_r1000_Trim_Fre...    5  2962.407025   

   Indices_Merger  Amplitude_Merger  Phase_Merger  Frequency_Merger  Validity  
0           18668          0.060509    147.046660          0.125639      True  
1            4227          0.007726     88.701075          0.051915      True  
2            3038          0.021498    118.053180          0.082941      True  
3            2739          0.020364     58.374650          0.085788      True  
4            3055          0.022009    147.306199          0.213374      True  


In [18]:
print(len(BAM_psi4_mrg_Lev1['File_Label']))
extract_psi4_mrg_Lev1 = [[],[],[]]
iden_list2 = []

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_psi4_mrg_Lev1['File_Label'])):
        
        id1 = BAM_psi4_mrg_Lev1['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_psi4_mrg_Lev1['Validity'][j] == True:
                #print(type(BAM_psi4_mrg_Lev1['Amplitude_Merger'][j]),type(BAMData['M0'][i]))
                #Add to list of merger times
                extract_psi4_mrg_Lev1[0].append(BAM_psi4_mrg_Lev1['Time_Merger'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_psi4_mrg_Lev1[1].append(abs(BAM_psi4_mrg_Lev1['Frequency_Merger'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of merger amplitudes
                extract_psi4_mrg_Lev1[2].append((BAM_psi4_mrg_Lev1['Amplitude_Merger'][j])*float(BAMData['M0'][i]))
                #Make new list of identifiers to make sure it includes only the runs used
                iden_list2.append(iden_list[i])
                flag = False
                
    if flag == True:
        extract_psi4_mrg_Lev1[0].append('-')
        #Add to list of merger frequencies
        extract_psi4_mrg_Lev1[1].append('-')  #Take absolute value
        #Add to list of merger amplitudes
        extract_psi4_mrg_Lev1[2].append('-')
                
        
        

print(extract_psi4_mrg_Lev1)
print(iden_list2)
print(len(extract_psi4_mrg_Lev1[0]))

69
[[34.187753808, 23.722853939999997, 29.433553632000006, 10.895150376000002, 14.583929783885834, '-', '-', '-', 25.227519660000006, 10.271347200000003, 13.79747451567366, 13.261566192653232, '-', '-', '-', 18.84244062139048, 12.301991702416355, '-', '-', '-', 52.94522399999999, 31.670879904000014, '-', '-', '-', 9.171549, 8.100796500000001, '-', '-', '-', '-', 9.821385000000001, '-', '-', 28.896050645999996, 9.2256389856, 12.692711994408658, '-', '-', '-', '-', 13.772401612865686, '-', '-', '-', 9.257603040000001, 10.060283170404718, 14.426386119475945, '-', '-', '-', 7.671563815711826, 14.889415219379178, '-', '-', '-', 40.52613599999999, 30.904900487999996, 7.216724160000001, 25.749101664000005, 7.590084479999999, '-', 35.650554336, 34.852122504, 27.063400881600003, '-', '-', '-', '-', 39.895992, '-', '-', '-', '-', '-', '-', '-', '-', '-', 21.310958088, 47.989404, '-', '-', '-', '-', '-', 13.863679992000002, 9.797759925575347, 38.527398000000005, 7.470523044666673, 38.232018000000

### Strain Merger Frequency

In [14]:
BAM_h_mrg_Lev1 = pd.read_csv('./Strain_Processed/Merger_Values_BAM_Lev1.txt', sep='\t')
print(BAM_h_mrg_Lev1.head())

                                          File_Label  EOS  Index_Merger  \
0   R01\BAM0001_2B_1350_1350_r850_Trim_Freq_Time.txt    0         16831   
1   R01\BAM0002_2H_1350_1350_r900_Trim_Freq_Time.txt    1          3611   
2  R01\BAM0003_ALF2_1350_1350_r900_Trim_Freq_Time...    2          2680   
3  R01\BAM0004_ALF2_1351_1351_r1000_Trim_Freq_Tim...    3          2465   
4  R01\BAM0005_ALF2_1375_1375_r1000_Trim_Freq_Tim...    4          2594   

   Time_Merger  Amplitude_Merger  Phase_Merger  Frequency_Merger  Validity  
0     6261.132          0.837606    133.068635          0.079478      True  
1     4116.540          0.551313     76.003966          0.038638      True  
2     5274.240          0.647520    104.343118          0.051045      True  
3     1991.720          0.657075     50.211876          0.051337      True  
4     6917.333          0.679791    127.949019          0.052458      True  


In [19]:
print(len(BAM_h_mrg_Lev1['File_Label']))
extract_h_mrg_Lev1 = [[],[],[]]

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_h_mrg_Lev1['File_Label'])):
        id1 = BAM_h_mrg_Lev1['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_h_mrg_Lev1['Validity'][j] == True:
            
                #Add to list of merger times
                extract_h_mrg_Lev1[0].append(BAM_h_mrg_Lev1['Time_Merger'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_h_mrg_Lev1[1].append(abs(BAM_h_mrg_Lev1['Frequency_Merger'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of merger amplitudes
                extract_h_mrg_Lev1[2].append(BAM_h_mrg_Lev1['Amplitude_Merger'][j]*float(BAMData['M0'][i]))
                #flag
                flag = False
                
    if flag == True:
        extract_h_mrg_Lev1[0].append('-')
        #Add to list of merger frequencies
        extract_h_mrg_Lev1[1].append('-')  
        #Add to list of merger amplitudes
        extract_h_mrg_Lev1[2].append('-')
        

print(extract_h_mrg_Lev1)
print(len(extract_h_mrg_Lev1[0]))
#print(iden_list2)
print('len=',len(iden_list))

69
[[30.823552836000005, 20.26572642, 25.96508352, 9.80523756, 34.054030359, '-', '-', '-', 22.025659536, 9.2197944, 37.375416, 32.806872, '-', '-', '-', 49.938897231, 30.32567999999999, '-', '-', '-', 29.892470769, 28.15089552, '-', '-', '-', 7.9974135, 6.934045500000001, '-', '-', '-', '-', 7.812801, '-', '-', 25.440281874000007, 8.10045189, 31.257766359000005, '-', '-', '-', '-', 34.198441641, '-', '-', '-', 8.19325044, 26.544816, 35.826310359, '-', '-', '-', 23.157792, 32.43928635900001, '-', '-', '-', 29.774304, 27.507321576000006, 6.2242473600000014, 22.305384396, 6.512538240000001, '-', 32.01367823999999, 31.150302191999987, 23.696235279, '-', '-', '-', '-', 36.016668, '-', '-', '-', '-', '-', '-', '-', '-', '-', 18.132551136, 26.062362, '-', '-', '-', '-', '-', 10.952651016, 25.744011282000006, 34.677612, 22.435750359000004, 29.78415, 26.873017641000004, 27.923256000000002, 22.832874, 30.264634800000003, 28.563954912000003, 28.860280128000003, 28.869259679999992, 12.243884994, 

In [20]:
BAM_h_mrg_Lev2 = pd.read_csv('./Strain_Processed/Merger_Values_BAM_Lev2.txt', sep='\t')
print(BAM_h_mrg_Lev2.head())

                                          File_Label  EOS  Index_Merger  \
0   R02\BAM0001_2B_1350_1350_r850_Trim_Freq_Time.txt    0         12654   
1   R02\BAM0002_2H_1350_1350_r900_Trim_Freq_Time.txt    1          1591   
2  R02\BAM0003_ALF2_1350_1350_r900_Trim_Freq_Time...    2          5235   
3  R02\BAM0004_ALF2_1351_1351_r1000_Trim_Freq_Tim...    3          2461   
4  R02\BAM0005_ALF2_1375_1375_r1000_Trim_Freq_Tim...    4          1659   

   Time_Merger  Amplitude_Merger  Phase_Merger  Frequency_Merger  Validity  
0     6276.384          0.832798    133.145858          0.079043      True  
1     4085.688          0.545613     75.279547          0.038267      True  
2     5276.880          0.643859    104.195628          0.050634      True  
3     1988.488          0.657504     50.128937          0.051309      True  
4     6636.000          0.671142    122.475538          0.051971      True  


In [22]:
print(len(BAM_h_mrg_Lev2['File_Label']))
extract_h_mrg_Lev2 = [[],[],[]]

for i in range(len(iden_list)):
    
    flag = True
    
    for j in range(len(BAM_h_mrg_Lev2['File_Label'])):
        
        id1 = BAM_h_mrg_Lev2['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_h_mrg_Lev2['Validity'][j] == True:
                
                #Add to list of merger times
                extract_h_mrg_Lev2[0].append(BAM_h_mrg_Lev2['Time_Merger'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_h_mrg_Lev2[1].append(abs(BAM_h_mrg_Lev2['Frequency_Merger'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of merger amplitudes
                extract_h_mrg_Lev2[2].append(BAM_h_mrg_Lev2['Amplitude_Merger'][j]*float(BAMData['M0'][i]))
                #flag
                flag = False
                #Make new list of identifiers to make sure it includes only the runs used
    if flag == True:
        
        extract_h_mrg_Lev2[0].append('-')
        #Add to list of merger frequencies
        extract_h_mrg_Lev2[1].append('-')  
        #Add to list of merger amplitudes
        extract_h_mrg_Lev2[2].append('-')


print(extract_h_mrg_Lev2)
print(len(extract_h_mrg_Lev2[2]))

61
[[30.89863843199999, 20.113842024, 25.978080239999997, 9.789326424, 32.669028000000004, '-', '-', '-', '-', 9.211917600000001, 36.4302, 31.526892, '-', '-', '-', 48.44232, 29.242620000000002, '-', '-', '-', 27.86418, 27.818061336000003, '-', '-', '-', 7.9871992596, '-', '-', '-', '-', '-', '-', '-', '-', 25.239748392000003, 8.096055651, 30.936132, '-', '-', '-', '-', 33.693012, '-', '-', '-', 8.18872128, 26.308512, 35.032068, '-', '-', '-', 23.170921641000003, 31.389048000000003, '-', '-', '-', 28.553400000000003, 27.103478040000002, 6.214795200000001, 21.964456799999997, 6.498360000000001, '-', '-', 30.519961272000007, 23.703683778000002, '-', '-', '-', '-', 35.747547282, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 31.717264410000002, '-', '-', '-', '-', '-', 10.96411176, 25.704627282000004, 34.27720471800001, 22.192884000000003, 29.27544164099999, 26.662968000000003, 26.702352, 22.278214359000003, 34.083962199000005, 25.385193504000004, 28.312527455999994, '-', 11.9909363310

In [23]:
BAM_h_mrg_Lev3 = pd.read_csv('./Strain_Processed/Merger_Values_BAM_Lev3.txt', sep='\t')
print(BAM_h_mrg_Lev3.head())

                                          File_Label  EOS  Index_Merger  \
0   R03\BAM0001_2B_1350_1350_r850_Trim_Freq_Time.txt    0          8543   
1   R03\BAM0026_G2_1515_1515_r400_Trim_Freq_Time.txt    1           269   
2   R03\BAM0035_H4_1350_1350_r900_Trim_Freq_Time.txt    2          2288   
3  R03\BAM0037_H4_1372_1372_r1000_Trim_Freq_Time.txt    3          2315   
4  R03\BAM0052_H4_1600_1600_r1000_Trim_Freq_Time.txt    4          1169   

   Time_Merger  Amplitude_Merger  Phase_Merger  Frequency_Merger  Validity  
0     6355.992          0.817245    133.928384          0.077105      True  
1     1614.000          0.693086     37.155814          0.042323      True  
2     5051.904          0.608128     97.008652          0.045430      True  
3     6173.334          0.639971    113.853065          0.047272      True  
4     4676.000          0.830725     89.640969          0.048992      True  


In [24]:
print(len(BAM_h_mrg_Lev3['File_Label']))
extract_h_mrg_Lev3 = [[],[],[]]

for i in range(len(iden_list)):
    #print(i)
    flag = True
    for j in range(len(BAM_h_mrg_Lev3['File_Label'])):
        
        id1 = BAM_h_mrg_Lev3['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_h_mrg_Lev3['Validity'][j] == True:
            
                #Add to list of merger times
                extract_h_mrg_Lev3[0].append(BAM_h_mrg_Lev3['Time_Merger'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_h_mrg_Lev3[1].append(abs(BAM_h_mrg_Lev3['Frequency_Merger'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of merger amplitudes
                extract_h_mrg_Lev3[2].append(BAM_h_mrg_Lev3['Amplitude_Merger'][j]*float(BAMData['M0'][i]))
                #flag
                flag = False
                #Make new list of identifiers to make sure it includes only the runs used
    if flag == True:
        extract_h_mrg_Lev3[0].append('-')
        #Add to list of merger frequencies
        extract_h_mrg_Lev3[1].append('-')  #Take absolute value
        #Add to list of merger amplitudes
        extract_h_mrg_Lev3[2].append('-')

print(extract_h_mrg_Lev3)
print(len(extract_h_mrg_Lev3[0]))

32
[[31.290548615999995, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 7.945722000000001, '-', '-', '-', '-', '-', '-', '-', '-', 24.870523392000003, '-', 30.391323282000005, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 23.019948000000003, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 29.538157536000003, 23.706548964, '-', '-', '-', '-', 34.9533, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 29.577384000000002, '-', '-', '-', '-', '-', 10.736427933000002, 25.284528, 33.220404, '-', 28.120176, '-', '-', 21.090132, 29.68155468, '-', 28.785450528000005, '-', 12.189783685500002, 6.0629954796000005, '-', '-', '-', '-', '-', '-', 35.8704549, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 32.515194096, '-', '-', 10.787907743999998, 32.715323892, '-', 33.64330939199999, 34.11638999999999, '-', '-', 9.137560608000003, '-', '-', '-', '-', '-', '-', '-', 10.311182474999999,

### $\Psi_4$ Merger Frequency

In [25]:
BAM_psi4_mrg_Lev2 = pd.read_csv('./Processed/Merger_Values_BAM_Lev2.txt', sep='\t')
print(BAM_psi4_mrg_Lev2.head())

                                          File_Label  EOS  Time_Merger  \
0  R02\BAM0001_2B_1350_1350_psi4_r850_Trim_Freq_T...    1  6954.912000   
1  R02\BAM0002_2H_1350_1350_psi4_r900_Trim_Freq_T...    2  4768.776000   
2  R02\BAM0003_ALF2_1350_1350_psi4_r900_Trim_Freq...    3  5996.592000   
3  R02\BAM0004_ALF2_1351_1351_psi4_r1000_Trim_Fre...    4  2222.808000   
4  R02\BAM0005_ALF2_1375_1375_psi4_r1000_Trim_Fre...    5  2862.528818   

   Indices_Merger  Amplitude_Merger  Phase_Merger  Frequency_Merger  Validity  
0           14022          0.058547    146.830110          0.121180      True  
1            1857          0.007513     86.811666          0.048981      True  
2            5949          0.018518    119.314094          0.084413      True  
3            2751          0.019638     59.301779          0.089620      True  
4            1968          0.020049    142.108484          0.207835      True  


In [27]:
print(len(BAM_psi4_mrg_Lev2['File_Label']))
extract_psi4_mrg_Lev2 = [[],[],[]]

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_psi4_mrg_Lev2['File_Label'])):
        
        id1 = BAM_psi4_mrg_Lev2['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_psi4_mrg_Lev2['Validity'][j] == True:
                #Add to list of merger times
                extract_psi4_mrg_Lev2[0].append(BAM_psi4_mrg_Lev2['Time_Merger'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_psi4_mrg_Lev2[1].append(abs(BAM_psi4_mrg_Lev2['Frequency_Merger'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of merger amplitudes
                extract_psi4_mrg_Lev2[2].append(BAM_psi4_mrg_Lev2['Amplitude_Merger'][j]*float(BAMData['M0'][i]))
                #flag
                flag = False
            
    if flag == True:
        #Add to list of merger times
        extract_psi4_mrg_Lev2[0].append('-')
        #Add to list of merger frequencies
        extract_psi4_mrg_Lev2[1].append('-')  
        #Add to list of merger amplitudes
        extract_psi4_mrg_Lev2[2].append('-')
        

print(extract_psi4_mrg_Lev2)
print(len(extract_psi4_mrg_Lev2[0]))

61
[[34.239031776, 23.476684248, 29.521222416, 10.942883784000001, 14.092229368913578, '-', '-', '-', '-', 10.247716800000003, 13.475839762453871, 12.817604473460737, '-', '-', '-', 18.356303191903272, 11.92247740659094, '-', '-', '-', 50.765976, 31.232535984, '-', '-', '-', 9.1687384593, '-', '-', '-', '-', '-', '-', '-', '-', 28.693685807999998, 9.194872204800003, 12.582652663260156, '-', '-', '-', '-', 13.64112222140856, '-', '-', '-', 9.25307388, 9.996832058629657, 14.135185076031862, '-', '-', '-', 7.7002808995139524, 14.51467360516909, '-', '-', '-', 39.147696, 30.54288276, 7.2309024000000015, 25.379049600000002, 7.594810559999999, '-', '-', 34.153017119999994, 27.116120304, '-', '-', '-', '-', 14.39775538231061, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 18.89893729858553, '-', '-', '-', '-', '-', 13.848399, 9.797759925575347, 13.872591326354673, 7.396679324291057, 13.705509584980591, 8.276293845151397, 37.355724, 39.58092, 13.732056734902937, 28.712117520000007, 31.70006

In [28]:
BAM_psi4_mrg_Lev3 = pd.read_csv('./Processed/Merger_Values_BAM_Lev3.txt', sep='\t')
print(BAM_psi4_mrg_Lev3.head())

                                          File_Label  EOS  Time_Merger  \
0  R03\BAM0001_2B_1350_1350_psi4_r850_Trim_Freq_T...    1  7040.472000   
1  R03\BAM0026_G2_1515_1515_psi4_r400_Trim_Freq_T...    2  1866.000000   
2  R03\BAM0035_H4_1350_1350_psi4_r900_Trim_Freq_T...    3  5747.424000   
3  R03\BAM0037_H4_1372_1372_psi4_r1000_Trim_Freq_...    4  2515.552986   
4  R03\BAM0052_H4_1600_1600_psi4_r1000_Trim_Freq_...    5  1543.727627   

   Indices_Merger  Amplitude_Merger  Phase_Merger  Frequency_Merger  Validity  
0            9463          0.054583    147.835606          0.118294      True  
1             311          0.015377     44.770446          0.063880      True  
2            2603          0.013318    109.708174          0.064221      True  
3            2588          0.015810    126.657270          0.186034      True  
4            1235          0.028295     96.158844          0.214555      True  


In [29]:
print(len(BAM_psi4_mrg_Lev3['File_Label']))
extract_psi4_mrg_Lev3 = [[],[],[]]

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_psi4_mrg_Lev3['File_Label'])):
        
        id1 = BAM_psi4_mrg_Lev3['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_psi4_mrg_Lev3['Validity'][j] == True:
            
                #Add to list of merger times
                extract_psi4_mrg_Lev3[0].append(BAM_psi4_mrg_Lev3['Time_Merger'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_psi4_mrg_Lev3[1].append(abs(BAM_psi4_mrg_Lev3['Frequency_Merger'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of merger amplitudes
                extract_psi4_mrg_Lev3[2].append(BAM_psi4_mrg_Lev3['Amplitude_Merger'][j]*float(BAMData['M0'][i]))
                #flag
                flag = False
            
    if flag == True:
        #Add to list of merger times
            extract_psi4_mrg_Lev3[0].append('-')
            #Add to list of merger frequencies
            extract_psi4_mrg_Lev3[1].append('-')  
            #Add to list of merger amplitudes
            extract_psi4_mrg_Lev3[2].append('-')
        

print(extract_psi4_mrg_Lev3)
print(len(extract_psi4_mrg_Lev3[0]))

32
[[34.660243656, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 9.186318, '-', '-', '-', '-', '-', '-', '-', '-', 28.294568352000002, '-', 12.384067348361773, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 7.599771106206501, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 33.113909664000005, 27.116120304, '-', '-', '-', '-', 14.106554096395172, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 18.069253471640113, '-', '-', '-', '-', '-', 13.6061952768, 9.701488948191397, 13.476367967756106, '-', 13.283030449393413, '-', '-', 38.49786, 12.28206672020899, '-', 32.182714368000006, '-', 13.220588502000002, 6.9878873664, '-', '-', '-', '-', '-', '-', 58.57474014, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 13.127124308766478, '-', '-', 11.820792528, 19.046838463796323, '-', 13.530679217652905, 13.71324709736819, '-', '-', 21.587308925536576, 21.03052830991418, 32.5784448,

### Strain Max Frequency

In [30]:
BAM_h_max_Lev1 = pd.read_csv('./Strain_Processed/Max_Frequencies_BAM_Lev1.txt', sep='\t')
print(BAM_h_max_Lev1.head())
#print(THC_h_max_Lev1)

                                          File_Label  EOS  Index_Max  \
0   R01\BAM0001_2B_1350_1350_r850_Trim_Freq_Time.txt    1      17098   
1   R01\BAM0002_2H_1350_1350_r900_Trim_Freq_Time.txt    2       3775   
2  R01\BAM0003_ALF2_1350_1350_r900_Trim_Freq_Time...    3       2754   
3  R01\BAM0004_ALF2_1351_1351_r1000_Trim_Freq_Tim...    4       2629   
4  R01\BAM0005_ALF2_1375_1375_r1000_Trim_Freq_Tim...    5       2645   

   Time_of_Max_Frequency  Max_Frequency  Min_Amplitude  Validity  
0               6360.456       0.094605       0.037901      True  
1               4303.500       0.012118       0.084322      True  
2               5419.872       0.143876       0.092707      True  
3               2124.232       0.245908       0.044642      True  
4               7053.333       0.160133       0.091107      True  


In [31]:
print(len(BAM_h_max_Lev1['File_Label']))
extract_h_max_Lev1 = [[],[],[]]

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_h_max_Lev1['File_Label'])):
        
        id1 = BAM_h_max_Lev1['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_h_max_Lev1['Validity'][j] == True:
            
                #Add to list of merger times
                extract_h_max_Lev1[0].append(BAM_h_max_Lev1['Time_of_Max_Frequency'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_h_max_Lev1[1].append(abs(BAM_h_max_Lev1['Max_Frequency'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of minimum amplitudes
                extract_h_max_Lev1[2].append(BAM_h_max_Lev1['Min_Amplitude'][j]*float(BAMData['M0'][i]))
                #flag
                flag = False
            
    if flag == True:
        #Add to list of merger times
        extract_h_max_Lev1[0].append('-')
        #Add to list of merger frequencies
        extract_h_max_Lev1[1].append('-')
        #Add to list of minimum amplitudes
        extract_h_max_Lev1[2].append('-')
            

print(extract_h_max_Lev1)
print(len(extract_h_max_Lev1[0]))

69
[[31.312524888000002, 21.1861305, 26.682029856, 10.457594136, 34.723558359, '-', '-', '-', 22.790575584, 9.9129528, 37.992435281999995, 33.463270359000006, '-', '-', '-', 50.621569641, 30.982083282000005, '-', '-', '-', 31.165853949, 28.734566400000002, '-', '-', '-', 8.758017, 7.702033500000001, '-', '-', '-', '-', 8.551251, '-', '-', 26.132534442000004, 8.856437616000001, 31.966678359000003, '-', '-', '-', '-', 34.85484, '-', '-', '-', 8.98585344, 27.240598358999996, 36.56148164099999, '-', '-', '-', 23.814190359, 33.226966359, '-', '-', '-', 31.218382359000003, 28.160702135999998, 7.12692864, 23.104416834, 7.415219519999999, '-', 32.812110072, 31.967835263999998, 24.52599216, '-', '-', '-', '-', 36.853578, '-', '-', '-', '-', '-', '-', '-', '-', '-', 19.082020608, 26.928810000000002, '-', '-', '-', '-', '-', 11.858049792000001, 26.531691282000004, 35.563752, 23.210300718, 30.591522, 27.581929641000002, 28.920985641000005, 24.41808, 30.796811100000006, 29.108113794, 29.39680836000

In [32]:
BAM_h_max_Lev2 = pd.read_csv('./Strain_Processed/Max_Frequencies_BAM_Lev2.txt', sep='\t')
print(BAM_h_max_Lev2.head())

                                          File_Label  EOS  Index_Max  \
0   R02\BAM0001_2B_1350_1350_r850_Trim_Freq_Time.txt    1      12865   
1   R02\BAM0002_2H_1350_1350_r900_Trim_Freq_Time.txt    2       1664   
2  R02\BAM0003_ALF2_1350_1350_r900_Trim_Freq_Time...    3       5377   
3  R02\BAM0004_ALF2_1351_1351_r1000_Trim_Freq_Tim...    4       2629   
4  R02\BAM0005_ALF2_1375_1375_r1000_Trim_Freq_Tim...    5       1691   

   Time_of_Max_Frequency  Max_Frequency  Min_Amplitude  Validity  
0               6381.040       0.101082       0.024411      True  
1               4273.152       0.006249       0.073245      True  
2               5420.016       0.157997       0.086116      True  
3               2124.232       0.177366       0.081735      True  
4               6764.000       0.223679       0.048888      True  


In [33]:
print(len(BAM_h_max_Lev2['File_Label']))
extract_h_max_Lev2 = [[],[],[]]

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_h_max_Lev2['File_Label'])):
        
        id1 = BAM_h_max_Lev2['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_h_max_Lev2['Validity'][j] == True:
            
                #Add to list of merger times
                extract_h_max_Lev2[0].append(BAM_h_max_Lev2['Time_of_Max_Frequency'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_h_max_Lev2[1].append(abs(BAM_h_max_Lev2['Max_Frequency'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of minimum amplitudes
                extract_h_max_Lev2[2].append(BAM_h_max_Lev2['Min_Amplitude'][j]*float(BAMData['M0'][i]))
                #flag
                flag = False
                
    if flag == True:
        #Add to list of merger times
        extract_h_max_Lev2[0].append('-')
        #Add to list of merger frequencies
        extract_h_max_Lev2[1].append('-')
        #Add to list of minimum amplitudes
        extract_h_max_Lev2[2].append('-')

print(extract_h_max_Lev2)
print(len(extract_h_max_Lev2[0]))

61
[[31.413859919999997, 21.036727296000002, 26.682738768, 10.457594136, 33.299172, '-', '-', '-', '-', 9.9011376, 37.080036, 32.19642, '-', '-', '-', 49.387536000000004, 29.93184, '-', '-', '-', 28.592784, 28.435523688000004, '-', '-', '-', 8.729883039599999, '-', '-', '-', '-', '-', '-', '-', '-', 25.964965368000005, 8.834461344, 31.625352000000003, '-', '-', '-', '-', 34.461, '-', '-', '-', 8.96320764, 27.056808, 35.800056000000005, '-', '-', '-', 23.801065641000005, 32.235804, '-', '-', '-', 29.912148000000002, 27.781670519999995, 7.112750400000001, 22.8210588, 7.43884992, '-', '-', 31.333674095999992, 24.511666230000003, '-', '-', '-', '-', 36.574611282, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 32.649336000000005, '-', '-', '-', '-', '-', 11.850409296, 26.505432000000003, 35.143652718000006, 23.039640000000002, 30.14188964099999, 27.430956000000002, 27.706644, 23.879833641000005, 34.14566708099999, 26.094578112, 28.855790352000007, '-', 12.540697587000002, 6.5981748096000

In [34]:
BAM_h_max_Lev3 = pd.read_csv('./Strain_Processed/Max_Frequencies_BAM_Lev3.txt', sep='\t')
print(BAM_h_max_Lev3.head())

                                          File_Label  EOS  Index_Max  \
0   R03\BAM0001_2B_1350_1350_r850_Trim_Freq_Time.txt    1       8699   
1   R03\BAM0026_G2_1515_1515_r400_Trim_Freq_Time.txt    2        294   
2   R03\BAM0035_H4_1350_1350_r900_Trim_Freq_Time.txt    3       2360   
3  R03\BAM0037_H4_1372_1372_r1000_Trim_Freq_Time.txt    4       2368   
4  R03\BAM0052_H4_1600_1600_r1000_Trim_Freq_Time.txt    5       1223   

   Time_of_Max_Frequency  Max_Frequency  Min_Ampliude  Validity  
0               6472.056      -0.363337      0.004424      True  
1               1764.000      -0.074606      0.037729      True  
2               5210.880       0.365260      0.014001      True  
3               6314.667      -0.434503      0.004579      True  
4               4892.000       0.168942      0.015584      True  


In [37]:
print(len(BAM_h_max_Lev3['File_Label']))
extract_h_max_Lev3 = [[],[],[]]

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_h_max_Lev3['File_Label'])):
        
        id1 = BAM_h_max_Lev3['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_h_max_Lev3['Validity'][j] == True:
            
                #Add to list of merger times
                extract_h_max_Lev3[0].append(BAM_h_max_Lev3['Time_of_Max_Frequency'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_h_max_Lev3[1].append(abs(BAM_h_max_Lev3['Max_Frequency'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of minimum amplitudes
                extract_h_max_Lev3[2].append(BAM_h_max_Lev3['Min_Ampliude'][j]*float(BAMData['M0'][i])) #Note the misspelling (to avoid rerunning another part)
                #flag
                flag = False
                
    if flag == True:
        #Add to list of merger times
        extract_h_max_Lev3[0].append('-')
        #Add to list of merger frequencies
        extract_h_max_Lev3[1].append('-')  
        #Add to list of minimum amplitudes
        extract_h_max_Lev3[2].append('-')

print(extract_h_max_Lev3)
print(len(extract_h_max_Lev3[0]))

32
[[31.861931688000006, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 8.684172, '-', '-', '-', '-', '-', '-', '-', '-', 25.65316224, '-', 31.087105641000004, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 24.083316, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 30.355690608000003, 24.52599216, '-', '-', '-', '-', 35.721288, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 30.5226, '-', '-', '-', '-', '-', 11.667037392, 26.170668000000003, 34.106544, '-', 29.006316, '-', '-', 22.114116000000003, 30.236869079999998, '-', 29.354155488000007, '-', 12.746860519500002, 6.621519675599999, '-', '-', '-', '-', '-', '-', 36.58448682, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 33.115642560000005, '-', '-', 11.354328432, 33.424974342000006, '-', 34.280148672, 34.80781550399999, '-', '-', 9.535240548, '-', '-', '-', '-', '-', '-', '-', 10.976033624999998, 10.780303350000002, 

### $\Psi_4$ Max Frequency

In [38]:
BAM_psi4_max_Lev1 = pd.read_csv('./Processed/Max_Frequencies_BAM_Lev1.txt', sep='\t')
print(BAM_psi4_max_Lev1.head())

                                          File_Label  EOS  Index_Max  \
0  R01\BAM0001_2B_1350_1350_psi4_r850_Trim_Freq_T...    1      18809   
1  R01\BAM0002_2H_1350_1350_psi4_r900_Trim_Freq_T...    2       4329   
2  R01\BAM0003_ALF2_1350_1350_psi4_r900_Trim_Freq...    3       3091   
3  R01\BAM0004_ALF2_1351_1351_psi4_r1000_Trim_Fre...    4       2819   
4  R01\BAM0005_ALF2_1375_1375_psi4_r1000_Trim_Fre...    5       3100   

   Time_of_Max_Frequency  Max_Frequency  Min_Amplitude  Validity  
0            6996.948000       0.750162       0.001962      True  
1            4935.060000       0.118767       0.000860      True  
2            6083.088000       0.092947       0.010682      True  
3            2277.752000       0.116299       0.010769      True  
4            3006.043135       0.240279       0.012838      True  


In [40]:
print(len(BAM_psi4_max_Lev1['File_Label']))
extract_psi4_max_Lev1 = [[],[],[]]

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_psi4_max_Lev1['File_Label'])):
        
        id1 = BAM_psi4_max_Lev1['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_psi4_max_Lev1['Validity'][j] == True:
                #Add to list of merger times
                extract_psi4_max_Lev1[0].append(BAM_psi4_max_Lev1['Time_of_Max_Frequency'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_psi4_max_Lev1[1].append(abs(BAM_psi4_max_Lev1['Max_Frequency'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of minimum amplitudes
                extract_psi4_max_Lev1[2].append(BAM_psi4_max_Lev1['Min_Amplitude'][j]*float(BAMData['M0'][i]))
                #flag
                flag = False
                
    if flag == True:
        #Add to list of merger times
        extract_psi4_max_Lev1[0].append('-')
        #Add to list of merger frequencies
        extract_psi4_max_Lev1[1].append('-')
        #Add to list of minimum amplitudes
        extract_psi4_max_Lev1[2].append('-')
        

print(extract_psi4_max_Lev1)
print(len(extract_psi4_max_Lev1[0]))

69
[[34.445975004, 24.295300380000004, 29.947042224, 11.213373096000003, 14.798750353533908, '-', '-', '-', 25.796303388000005, 10.657310400000002, 13.889370159450742, 13.404779650457261, '-', '-', '-', 18.94089883495752, 12.46429970314672, '-', '-', '-', 53.43096, 31.976184672000013, '-', '-', '-', 9.5776965, 8.492175000000001, '-', '-', '-', '-', 11.0545965, '-', '-', 29.286129474, 9.608026118400002, 12.704674965185665, '-', '-', '-', '-', 13.92038929050826, '-', '-', '-', 9.88262712, 10.17843351646863, 14.579147322594158, '-', '-', '-', 7.770022388747696, 15.05172368922813, '-', '-', '-', 41.038127999999986, 30.951570528000005, 7.85001888, 26.244276695999996, 8.1288576, '-', 36.242692776, 35.31055226400001, 27.650190974400005, '-', '-', '-', '-', 40.437522, '-', '-', '-', '-', '-', '-', '-', '-', '-', 22.011481296000003, 48.058326, '-', '-', '-', '-', '-', 14.429076696000003, 9.924662577581458, 39.108312000000005, 7.626415343237421, 38.734164, 8.45197444694957, 39.25272000000002, 13

In [41]:
BAM_psi4_max_Lev2 = pd.read_csv('./Processed/Max_Frequencies_BAM_Lev2.txt', sep='\t')
print(BAM_psi4_max_Lev2.head())

                                          File_Label  EOS  Index_Max  \
0  R02\BAM0001_2B_1350_1350_psi4_r850_Trim_Freq_T...    1      14144   
1  R02\BAM0002_2H_1350_1350_psi4_r900_Trim_Freq_T...    2       1869   
2  R02\BAM0003_ALF2_1350_1350_psi4_r900_Trim_Freq...    3       6029   
3  R02\BAM0004_ALF2_1351_1351_psi4_r1000_Trim_Fre...    4       2831   
4  R02\BAM0005_ALF2_1375_1375_psi4_r1000_Trim_Fre...    5       1986   

   Time_of_Max_Frequency  Max_Frequency  Min_Amplitude  Validity  
0            7015.424000       0.592534       0.001881      True  
1            4799.592000       0.049588       0.007401      True  
2            6077.232000       0.094028       0.010896      True  
3            2287.448000       0.104290       0.012811      True  
4            2888.710484       0.318683       0.012155      True  


In [42]:
print(len(BAM_psi4_max_Lev2['File_Label']))
extract_psi4_max_Lev2 = [[],[],[]]

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_psi4_max_Lev2['File_Label'])):
        
        id1 = BAM_psi4_max_Lev2['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_psi4_max_Lev2['Validity'][j] == True:
            
                #Add to list of merger times
                extract_psi4_max_Lev2[0].append(BAM_psi4_max_Lev2['Time_of_Max_Frequency'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_psi4_max_Lev2[1].append(abs(BAM_psi4_max_Lev2['Max_Frequency'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of minimum amplitudes
                extract_psi4_max_Lev2[2].append(BAM_psi4_max_Lev2['Min_Amplitude'][j]*float(BAMData['M0'][i]))
                #flag
                flag = False
                
    if flag == True:
        #Add to list of merger times
        extract_psi4_max_Lev2[0].append('-')
        #Add to list of merger frequencies
        extract_psi4_max_Lev2[1].append('-')
        #Add to list of minimum amplitudes
        extract_psi4_max_Lev2[2].append('-')

print(extract_psi4_max_Lev2)
print(len(extract_psi4_max_Lev2[0]))

61
[[34.536932351999994, 23.628391416, 29.918213136, 11.261106504000002, 14.221121710702423, '-', '-', '-', '-', 10.7282016, 13.587427329897471, 12.953657258374564, '-', '-', '-', 18.5162977889497, 12.072850995502897, '-', '-', '-', 51.218892000000004, 31.55969887200001, '-', '-', '-', 9.5468307669, '-', '-', '-', '-', '-', '-', '-', '-', 29.092921416000003, 9.572864083200002, 12.736974986283597, '-', '-', '-', '-', 13.784336102998152, '-', '-', '-', 9.77392728, 10.12811022092289, 14.299880748143682, '-', '-', '-', 7.765919948204537, 14.686529632067982, '-', '-', '-', 39.777840000000005, 31.066059816, 7.87364928, 25.946179200000003, 8.14776192, '-', '-', 34.6687506, 27.646179714000002, '-', '-', '-', '-', 14.603028419923126, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 19.15624430820783, '-', '-', '-', '-', '-', 14.440537440000002, 9.924662577581458, 14.05399479173715, 7.538213121677655, 13.906008157801619, 8.409502213547814, 38.025252, 39.64656000000002, 13.748053472653693, 29.14

In [43]:
BAM_psi4_max_Lev3 = pd.read_csv('./Processed/Max_Frequencies_BAM_Lev3.txt', sep='\t')
print(BAM_psi4_max_Lev3.head())

                                          File_Label  EOS  Index_Max  \
0  R03\BAM0001_2B_1350_1350_psi4_r850_Trim_Freq_T...    1       9561   
1  R03\BAM0026_G2_1515_1515_psi4_r400_Trim_Freq_T...    2        322   
2  R03\BAM0035_H4_1350_1350_psi4_r900_Trim_Freq_T...    3       2654   
3  R03\BAM0037_H4_1372_1372_psi4_r1000_Trim_Freq_...    4       2621   
4  R03\BAM0052_H4_1600_1600_psi4_r1000_Trim_Freq_...    5       1257   

   Time_of_Max_Frequency  Max_Frequency  Min_Amplitude  Validity  
0            7113.384000       0.300919       0.003938      True  
1            1932.000000       0.115657       0.008252      True  
2            5860.032000       0.087801       0.004854      True  
3            2547.629202       0.291831       0.006349      True  
4            1571.227228       0.113445       0.004477      True  


In [45]:
print(len(BAM_psi4_max_Lev3['File_Label']))
extract_psi4_max_Lev3 = [[],[],[]]

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_psi4_max_Lev3['File_Label'])):
        
        id1 = BAM_psi4_max_Lev3['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            if BAM_psi4_max_Lev3['Validity'][j] == True:
                #Add to list of merger times
                extract_psi4_max_Lev3[0].append(BAM_psi4_max_Lev3['Time_of_Max_Frequency'][j]*(4.923e-3))
                #Add to list of merger frequencies
                extract_psi4_max_Lev3[1].append(abs(BAM_psi4_max_Lev3['Max_Frequency'][j])/(2*np.pi*(4.923e-3)))  #Take absolute value
                #Add to list of minimum amplitudes
                extract_psi4_max_Lev3[2].append(BAM_psi4_max_Lev3['Min_Amplitude'][j]*float(BAMData['M0'][i]))
                #flag
                flag = False
                
    if flag == True:
        #Add to list of merger times
        extract_psi4_max_Lev3[0].append('-')
        #Add to list of merger frequencies
        extract_psi4_max_Lev3[1].append('-')
        #Add to list of minimum amplitudes
        extract_psi4_max_Lev3[2].append('-')

print(extract_psi4_max_Lev3)
print(len(extract_psi4_max_Lev3[0]))

32
[[35.019189432000005, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 9.511236, '-', '-', '-', '-', '-', '-', '-', '-', 28.848937536, '-', 12.541978562618317, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 7.735151644130825, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 33.68694686399999, 27.671966387999998, '-', '-', '-', '-', 14.285571280359582, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 18.32130931780075, '-', '-', '-', '-', '-', 14.261749833600001, 9.832767553714964, 13.6911878609722, '-', 13.490689685529482, '-', '-', 39.246156, 12.371191401963229, '-', 32.470060032, '-', 13.422362580000001, 7.176457958400003, '-', '-', '-', '-', '-', '-', 59.00894874, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 13.239604708589578, '-', '-', 12.083641344, 19.221513329341274, '-', 13.67624153686482, 13.865542936691938, '-', '-', 21.76345032575335, 21.192147947215137, 32.

### Strain $f_2$ Frequency

In [46]:
BAM_h_f2_Lev1 = pd.read_csv('./Strain_Processed/f2_BAM_Lev1.txt', sep='\t')
print(BAM_h_f2_Lev1.head())

   Unnamed: 0                        File_Label    frequency  index
0           0     R01\BAM0001_2B_1350_1350_r850   764.073380     14
1           1     R01\BAM0002_2H_1350_1350_r900  1852.161970    104
2           2   R01\BAM0003_ALF2_1350_1350_r900  1021.317034     99
3           3  R01\BAM0004_ALF2_1351_1351_r1000  2914.721760    116
4           4  R01\BAM0005_ALF2_1375_1375_r1000  2847.431890    374


In [47]:
print(len(BAM_h_f2_Lev1['File_Label']))
extract_h_f2_Lev1 = []

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_h_f2_Lev1['File_Label'])):
        
        id1 = BAM_h_f2_Lev1['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            #Add to list of f2 frequencies
            extract_h_f2_Lev1.append(BAM_h_f2_Lev1['frequency'][j]/1000) #Divide by 1000 to express in kHz
            #flag
            flag = False
            
    if flag == True:
        extract_h_f2_Lev1.append('-')
            
            
print(extract_h_f2_Lev1)
print(len(extract_h_f2_Lev1))

69
[0.7640733803516875, 1.852161970150938, 1.0213170338455426, 2.914721759957637, 2.847431890361373, '-', '-', '-', 2.384938536380158, 2.6900871691471075, 0.7613462456465265, 2.7789093888942507, '-', '-', '-', 0.7308902415602156, 2.824591398462573, '-', '-', '-', 2.7408435308291317, 2.80495506423411, '-', '-', '-', 0.12181526803685018, 2.991241581793765, '-', '-', '-', '-', 2.23327991400892, '-', '-', 2.0011468763520788, 2.61513488470255, 1.0049762420236348, '-', '-', '-', '-', 2.5885758116419804, '-', '-', '-', 2.361274217380972, 0.9136135350270856, 2.4972128799516, '-', '-', '-', 0.7841860610325089, 2.588574327041124, '-', '-', '-', 2.443917324795816, 0.899479827276322, 2.072551435349187, 1.047987870249951, 1.903363563075784, '-', 1.22966307348253, 1.6221122772297016, 1.8314167452685455, '-', '-', '-', '-', 2.121615918308474, '-', '-', '-', '-', '-', '-', '-', '-', '-', 2.071686857250096, 1.9998006502716237, '-', '-', '-', '-', '-', 1.6221105279821106, 2.1774479717966764, 2.009951922

In [49]:
BAM_h_f2_Lev2 = pd.read_csv('./Strain_Processed/f2_BAM_Lev2.txt', sep='\t')
print(BAM_h_f2_Lev2.head())

   Unnamed: 0                        File_Label    frequency  index
0           0     R02\BAM0001_2B_1350_1350_r850   777.718445     19
1           1     R02\BAM0002_2H_1350_1350_r900  1810.467904    229
2           2   R02\BAM0003_ALF2_1350_1350_r900  2074.563030    103
3           3  R02\BAM0004_ALF2_1351_1351_r1000  2839.341025    113
4           4  R02\BAM0005_ALF2_1375_1375_r1000  2801.751165    552


In [50]:
print(len(BAM_h_f2_Lev2['File_Label']))
extract_h_f2_Lev2 = []

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_h_f2_Lev2['File_Label'])):
        
        id1 = BAM_h_f2_Lev2['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            #Add to list of f2 frequencies
            extract_h_f2_Lev2.append(BAM_h_f2_Lev2['frequency'][j]/1000) #Divide by 1000 to express in kHz
            #flag
            flag = False
            
    if flag == True:
        extract_h_f2_Lev2.append('-')
            
            
print(extract_h_f2_Lev2)
print(len(extract_h_f2_Lev2))

61
[0.7777184451277389, 1.810467903923667, 2.0745630295344832, 2.8393410247863184, 2.8017511648475537, '-', '-', '-', '-', 2.6900871691471075, 0.7258159720528989, 2.806826801015756, '-', '-', '-', 0.8273286954169407, 2.8119024371839583, '-', '-', '-', 2.705314077651714, 2.819799997644301, '-', '-', '-', 3.093974576885304, '-', '-', '-', '-', '-', '-', '-', '-', 2.0466299682125197, 2.706096225666065, 2.5276668117646413, '-', '-', '-', '-', 2.6850115329789053, '-', '-', '-', 2.339206234040963, 0.7511941528939093, 2.446456633073408, '-', '-', '-', 0.5633957299314244, 2.5479693564374495, '-', '-', '-', 2.497212994755428, 1.9601059877956073, 1.903363563075784, 1.9625793183714748, 2.030254467280836, '-', '-', 1.9447884287097368, 1.7093207714220096, '-', '-', '-', '-', 2.070860687457845, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 1.8957523422734601, '-', '-', '-', '-', '-', 1.936067404365746, 2.1850614372733452, 2.009951029657247, 1.1978501356956932, 1.9794981904829567, 1.2790603143869

In [51]:
BAM_h_f2_Lev3 = pd.read_csv('./Strain_Processed/f2_BAM_Lev3.txt', sep='\t')
print(BAM_h_f2_Lev3.head())

   Unnamed: 0                      File_Label    frequency  index
0           0   R03\BAM0001_2B_1350_1350_r850   736.784585     27
1           1   R03\BAM0026_G2_1515_1515_r400  2913.415161    861
2           2   R03\BAM0035_H4_1350_1350_r900  2381.503202    259
3           3  R03\BAM0037_H4_1372_1372_r1000   753.732108     99
4           4  R03\BAM0052_H4_1600_1600_r1000   756.269789    149


In [52]:
print(len(BAM_h_f2_Lev3['File_Label']))
extract_h_f2_Lev3 = []

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_h_f2_Lev3['File_Label'])):
        
        id1 = BAM_h_f2_Lev3['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            #Add to list of f2 frequencies
            extract_h_f2_Lev3.append(BAM_h_f2_Lev3['frequency'][j]/1000) #Divide by 1000 to express in kHz
            #flag
            flag = False
            
    if flag == True:
        extract_h_f2_Lev3.append('-')
            
            
print(extract_h_f2_Lev3)
print(len(extract_h_f2_Lev3))

32
[0.736784585461076, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 2.9134151605480003, '-', '-', '-', '-', '-', '-', '-', '-', 2.3815032021093137, '-', 0.7537321077218625, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 0.7562697890621115, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 2.014556623461655, 2.023277647805644, '-', '-', '-', '-', 1.9744224694306132, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 1.8069264758799441, '-', '-', '-', '-', '-', 1.8750208759835945, 2.23327991400892, 2.157145371485888, '-', 1.974422469430613, '-', '-', 2.086086465131059, 0.9719303300812504, '-', 2.304071681618053, '-', 2.643204014959804, 3.0053108890670264, '-', '-', '-', '-', '-', '-', 2.7139115838141787, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 3.6089757406029617, '-', '-', 3.536746478906776, 2.9497990126615417, '-', 3.4826349981971214, 3.4112218066085815, '-', '-', 2.

### $\Psi_4$ $f_2$ Data

In [53]:
BAM_psi4_f2_Lev1 = pd.read_csv('./Processed/f2_BAM_Lev1.txt', sep='\t')
print(BAM_psi4_f2_Lev1.head())

   Unnamed: 0                             File_Label    frequency  index
0           0     R01\BAM0001_2B_1350_1350_psi4_r850  2183.069320     40
1           1     R01\BAM0002_2H_1350_1350_psi4_r900  1852.161970    104
2           2   R01\BAM0003_ALF2_1350_1350_psi4_r900   969.735365     94
3           3  R01\BAM0004_ALF2_1351_1351_psi4_r1000  2914.721760    116
4           4  R01\BAM0005_ALF2_1375_1375_psi4_r1000  7788.608967    372


In [54]:
print(len(BAM_psi4_f2_Lev1['File_Label']))
extract_psi4_f2_Lev1 = []

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_psi4_f2_Lev1['File_Label'])):
        
        id1 = BAM_psi4_f2_Lev1['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            #Add to list of f2 frequencies
            extract_psi4_f2_Lev1.append(BAM_psi4_f2_Lev1['frequency'][j]/1000) #Divide by 1000 to express in kHz
            #flag
            flag = False
            
    if flag == True:
        extract_psi4_f2_Lev1.append('-')
            
            
print(extract_psi4_f2_Lev1)
print(len(extract_psi4_f2_Lev1))

69
[2.183069319656812, 1.852161970150938, 0.9697353654695064, 2.914721759957636, 7.788608967114865, '-', '-', '-', 2.2830170716411007, 2.6900871691471075, 2.28404512949541, 7.746748534813961, '-', '-', '-', 2.2658050956715288, 7.746776445639727, '-', '-', '-', 1.02020286980862, 2.7604337055134187, '-', '-', '-', 3.789808338924228, 3.1671969689581045, '-', '-', '-', '-', 2.8152861946294263, '-', '-', 1.9647623876911315, 2.6378754279186487, 2.882440583038607, '-', '-', '-', '-', 7.0348643389435175, '-', '-', '-', 2.361274217380972, 2.809403946191907, 6.846442599420243, '-', '-', '-', 2.558157703720768, 7.055827342175273, '-', '-', '-', 2.3677842724662743, 0.9101879204581812, 2.0725514353491867, 1.0213441108368164, 1.903363563075784, '-', 1.1773382864386284, 1.6221105279821113, 1.8314151122378681, '-', '-', '-', '-', 2.121615918308474, '-', '-', '-', '-', '-', '-', '-', '-', '-', 2.0889523005185473, 1.9591955609260068, '-', '-', '-', '-', '-', 1.6221105279821109, 6.623775558270725, 2.0404

In [55]:
BAM_psi4_f2_Lev2 = pd.read_csv('./Processed/f2_BAM_Lev2.txt', sep='\t')
print(BAM_psi4_f2_Lev2.head())

   Unnamed: 0                             File_Label    frequency  index
0           0     R02\BAM0001_2B_1350_1350_psi4_r850  1841.964738     45
1           1     R02\BAM0002_2H_1350_1350_psi4_r900  1810.468353    229
2           2   R02\BAM0003_ALF2_1350_1350_psi4_r900  2014.141337    100
3           3  R02\BAM0004_ALF2_1351_1351_psi4_r1000  2839.341025    113
4           4  R02\BAM0005_ALF2_1375_1375_psi4_r1000  7690.902403    551


In [56]:
print(len(BAM_psi4_f2_Lev2['File_Label']))
extract_psi4_f2_Lev2 = []

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_psi4_f2_Lev2['File_Label'])):
        
        id1 = BAM_psi4_f2_Lev2['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            #Add to list of f2 frequencies
            extract_psi4_f2_Lev2.append(BAM_psi4_f2_Lev2['frequency'][j]/1000) #Divide by 1000 to express in kHz
            #flag
            flag = False
            
    if flag == True:
        extract_psi4_f2_Lev2.append('-')
            
            
print(extract_psi4_f2_Lev2)
print(len(extract_psi4_f2_Lev2))

61
[1.8419647384604343, 1.8104683528322096, 2.0141413365881315, 2.8393410247863167, 7.6909024030112745, '-', '-', '-', '-', 2.6900871691471075, 2.1774563567856258, 7.704874218409562, '-', '-', '-', 2.5825305391524966, 7.718860134123908, '-', '-', '-', 2.705314077651714, 2.8848701297900763, '-', '-', '-', 3.1679934084224652, '-', '-', '-', '-', '-', '-', '-', '-', 1.992050754186841, 2.706096344157924, 6.851018197367099, '-', '-', '-', '-', 7.411732071386917, '-', '-', '-', 2.405410184060992, 2.4515665871918544, 6.727798843874737, '-', '-', '-', 18.321281839981093, 6.965099591979151, '-', '-', '-', 2.4413809969052056, 1.877572728887764, 1.903363563075784, 1.979498105598815, 2.030254467280836, '-', '-', 1.909904331333776, 1.6744366740460506, '-', '-', '-', '-', 5.715832640173039, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 4.872644625772008, '-', '-', '-', '-', '-', 1.936067404365746, 6.623775558270725, 5.548354503812458, 7.487708226215504, 5.4017940393808255, 8.214546462907892, 2.0

In [57]:
BAM_psi4_f2_Lev3 = pd.read_csv('./Processed/f2_BAM_Lev3.txt', sep='\t')
print(BAM_psi4_f2_Lev3.head())

   Unnamed: 0                           File_Label    frequency  index
0           0   R03\BAM0001_2B_1350_1350_psi4_r850  1446.283424     53
1           1   R03\BAM0026_G2_1515_1515_psi4_r400  2913.415161    861
2           2   R03\BAM0035_H4_1350_1350_psi4_r900  2344.723230    255
3           3  R03\BAM0037_H4_1372_1372_psi4_r1000  2026.063308     97
4           4  R03\BAM0052_H4_1600_1600_psi4_r1000  2420.098399    149


In [58]:
print(len(BAM_psi4_f2_Lev3['File_Label']))
extract_psi4_f2_Lev3 = []

for i in range(len(iden_list)):
    
    flag = True
    for j in range(len(BAM_psi4_f2_Lev3['File_Label'])):
        
        id1 = BAM_psi4_f2_Lev3['File_Label'][j].split('\\')
        id2 = id1[1].split('_')
        id3 = id2[0]
        #print(id3)
        if iden_list[i]==id3:
            
            #Add to list of f2 frequencies
            extract_psi4_f2_Lev3.append(BAM_psi4_f2_Lev3['frequency'][j]/1000) #Divide by 1000 to express in kHz
            #flag
            flag = False
            
    if flag == True:
        extract_psi4_f2_Lev3.append('-')
            
            
print(extract_psi4_f2_Lev3)
print(len(extract_psi4_f2_Lev3))

32
[1.446283424272639, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 2.9134151605480003, '-', '-', '-', '-', '-', '-', '-', '-', 2.3447232298759646, '-', 2.02606330836771, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 2.4200983990755125, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 2.040719696493624, 2.040719696493624, '-', '-', '-', '-', 5.429692056232751, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 4.403145013392925, '-', '-', '-', '-', '-', 1.7878099905179181, 6.517186066528439, 5.946161053142405, '-', 5.429710287646358, '-', '-', 2.0099519226080282, 2.799154689144567, '-', 2.304071681618054, '-', 3.919234212932703, 2.9496569837139344, '-', '-', '-', '-', '-', '-', 2.030254467280836, '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 9.970026577640388, '-', '-', 3.536746478906776, 7.319616772693988, '-', 9.516879585321416, 9.37371420151339, '-', '-', 2.146620178

### Simulation Parameter Lists

Note that we added this down at the bottom, as we needed to have this after we had a list of the runs used (i.e. a list of the runs for which a merger frequency was identified.

In [60]:
#
parameter_data = [[],[],[]] #EoS_q_M, Tidal Def., and M_chirp
print(len(iden_list))

for i in range(len(iden_list)):
    
    for j in range(len(BAMData['Model name'])):
        
        id1 = BAMData['Model name'][j].replace('_', '')
        id3 = id1
        #print(id3)
        if iden_list[i]==id3:
            print(i, iden_list[i], ',', BAMData['Model name'][j])
            #Add to list of f2 frequencies
            q_str = str((1/BAMData['q'][j])+0.000001)
            #print(q_str[:4])
            #print(BAMData['M0'][j])
            M_str = str(float(BAMData['M0'][j])+0.000001)
            run_id = str(BAMData['EOS name'][j])+'_'+q_str[:4]+'_'+M_str[:4]
            parameter_data[0].append(run_id.replace('.',''))
            parameter_data[1].append(BAMData['Lambda'][j])
            parameter_data[2].append(BAMData['M_chirp'][j])
            
print('len=',len(parameter_data))            
print(parameter_data)

147
0 BAM0001 , BAM_0001
1 BAM0002 , BAM_0002
2 BAM0003 , BAM_0003
3 BAM0004 , BAM_0004
4 BAM0005 , BAM_0005
5 BAM0006 , BAM_0006
6 BAM0007 , BAM_0007
7 BAM0008 , BAM_0008
8 BAM0009 , BAM_0009
9 BAM0010 , BAM_0010
10 BAM0011 , BAM_0011
11 BAM0012 , BAM_0012
12 BAM0013 , BAM_0013
13 BAM0014 , BAM_0014
14 BAM0015 , BAM_0015
15 BAM0016 , BAM_0016
16 BAM0017 , BAM_0017
17 BAM0018 , BAM_0018
18 BAM0019 , BAM_0019
19 BAM0020 , BAM_0020
20 BAM0021 , BAM_0021
21 BAM0022 , BAM_0022
22 BAM0023 , BAM_0023
23 BAM0024 , BAM_0024
24 BAM0025 , BAM_0025
25 BAM0026 , BAM_0026
26 BAM0027 , BAM_0027
27 BAM0028 , BAM_0028
28 BAM0029 , BAM_0029
29 BAM0030 , BAM_0030
30 BAM0031 , BAM_0031
31 BAM0032 , BAM_0032
32 BAM0033 , BAM_0033
33 BAM0034 , BAM_0034
34 BAM0035 , BAM_0035
35 BAM0036 , BAM_0036
36 BAM0037 , BAM_0037
37 BAM0038 , BAM_0038
38 BAM0039 , BAM_0039
39 BAM0040 , BAM_0040
40 BAM0041 , BAM_0041
41 BAM0042 , BAM_0042
42 BAM0043 , BAM_0043
43 BAM0044 , BAM_0044
44 BAM0045 , BAM_0045
45 BAM0046 , BAM

In [61]:
#Need to compute the time differences
t_diff_h_1 = []
t_diff_h_2 = []
t_diff_h_3 = []
t_diff_psi4_1 = []
t_diff_psi4_2 = []
t_diff_psi4_3 = []
for i in range(len(iden_list)):
    
    # h 1
    if extract_h_max_Lev1[0][i] == '-' or extract_h_mrg_Lev1[0][i] == '-':
        t_diff_h_1.append('-')
    else:
        t_diff_h_1.append(extract_h_max_Lev1[0][i]-extract_h_mrg_Lev1[0][i])
        
    # h 2
    if extract_h_max_Lev2[0][i] == '-' or extract_h_mrg_Lev2[0][i] == '-':
        t_diff_h_2.append('-')
    else:
        t_diff_h_2.append(extract_h_max_Lev2[0][i]-extract_h_mrg_Lev2[0][i])
        
    # h 3
    if extract_h_max_Lev3[0][i] == '-' or extract_h_mrg_Lev3[0][i] == '-':
        t_diff_h_3.append('-')
    else:
        t_diff_h_3.append(extract_h_max_Lev3[0][i]-extract_h_mrg_Lev3[0][i])
        
    # psi4 1
    if extract_psi4_max_Lev1[0][i] == '-' or extract_psi4_mrg_Lev1[0][i] == '-':
        t_diff_psi4_1.append('-')
    else:
        t_diff_psi4_1.append(extract_psi4_max_Lev1[0][i]-extract_psi4_mrg_Lev1[0][i])
        
    # psi4 2
    if extract_psi4_max_Lev2[0][i] == '-' or extract_psi4_mrg_Lev2[0][i] == '-':
        t_diff_psi4_2.append('-')
    else:
        t_diff_psi4_2.append(extract_psi4_max_Lev2[0][i]-extract_psi4_mrg_Lev2[0][i])
        
    # psi4 3
    if extract_psi4_max_Lev3[0][i] == '-' or extract_psi4_mrg_Lev3[0][i] == '-':
        t_diff_psi4_3.append('-')
    else:
        t_diff_psi4_3.append(extract_psi4_max_Lev3[0][i]-extract_psi4_mrg_Lev3[0][i])
        

print(t_diff_h_1)
print()
print(t_diff_h_2)
print()
print(t_diff_h_3)
print()
print(t_diff_psi4_1)
print()
print(t_diff_psi4_2)
print()
print(t_diff_psi4_3)
    

[0.4889720519999976, 0.9204040800000008, 0.7169463359999995, 0.6523565760000007, 0.6695279999999997, '-', '-', '-', 0.7649160479999999, 0.6931583999999997, 0.617019281999994, 0.6563983590000078, '-', '-', '-', 0.6826724100000021, 0.6564032820000136, '-', '-', '-', 1.2733831799999997, 0.5836708800000032, '-', '-', '-', 0.7606035000000002, 0.7679879999999999, '-', '-', '-', '-', 0.7384500000000003, '-', '-', 0.6922525679999971, 0.7559857260000005, 0.708911999999998, '-', '-', '-', '-', 0.6563983590000007, '-', '-', '-', 0.7926029999999997, 0.6957823589999954, 0.735171281999996, '-', '-', '-', 0.6563983590000007, 0.7876799999999946, '-', '-', '-', 1.4440783590000024, 0.6533805599999916, 0.9026812799999986, 0.7990324379999976, 0.9026812799999977, '-', 0.7984318320000128, 0.8175330720000105, 0.8297568810000016, '-', '-', '-', '-', 0.836909999999996, '-', '-', '-', '-', '-', '-', '-', '-', '-', 0.9494694720000005, 0.8664480000000019, '-', '-', '-', '-', '-', 0.9053987760000002, 0.78767999999

### Create Dataframe

In [63]:
#Check Lengths
print(len(extract_h_mrg_Lev1[1]))
print(len(parameter_data[0]))

#Create Dataframe
Data_Summary_BAM = pd.DataFrame({
    'Run Name': parameter_data[0],
    'Code': ['BAM' for i in range(len(parameter_data[0]))],
    'Identifier': iden_list,
    'M_chirp' : parameter_data[2],
    'Λ': parameter_data[1], 
    'fmrg,h [R01]': extract_h_mrg_Lev1[1],
    'fmrg,h [R02]': extract_h_mrg_Lev2[1],
    'fmrg,h [R03]': extract_h_mrg_Lev3[1],
    'fmrg,psi4 [R01]': extract_psi4_mrg_Lev1[1],
    'fmrg,psi4 [R02]': extract_psi4_mrg_Lev2[1],
    'fmrg,psi4 [R03]': extract_psi4_mrg_Lev3[1],
    'fmax,h [R01]': extract_h_max_Lev1[1],
    'fmax,h [R02]': extract_h_max_Lev2[1],
    'fmax,h [R03]': extract_h_max_Lev3[1],
    'fmax,psi4 [R01]': extract_psi4_max_Lev1[1],
    'fmax,psi4 [R02]': extract_psi4_max_Lev2[1],
    'fmax,psi4 [R03]': extract_psi4_max_Lev3[1],
    'f2,h [R01]': extract_h_f2_Lev1,
    'f2,h [R02]': extract_h_f2_Lev2,
    'f2,h [R03]': extract_h_f2_Lev3,
    'f2,psi4 [R01]': extract_psi4_f2_Lev1,
    'f2,psi4 [R02]': extract_psi4_f2_Lev2,
    'f2,psi4 [R03]': extract_psi4_f2_Lev3,
    'A_mrg,h [R01]': extract_h_mrg_Lev1[2],
    'A_mrg,h [R02]': extract_h_mrg_Lev2[2],
    'A_mrg,h [R03]': extract_h_mrg_Lev3[2],
    'A_mrg,psi4 [R01]': extract_psi4_mrg_Lev1[2],
    'A_mrg,psi4 [R02]': extract_psi4_mrg_Lev2[2],
    'A_mrg,psi4 [R03]': extract_psi4_mrg_Lev3[2],
    'A_max,h [R01]': extract_h_max_Lev1[2],
    'A_max,h [R02]': extract_h_max_Lev2[2],
    'A_max,h [R03]': extract_h_max_Lev3[2],
    'A_max,psi4 [R01]': extract_psi4_max_Lev1[2],
    'A_max,psi4 [R02]': extract_psi4_max_Lev2[2],
    'A_max,psi4 [R03]': extract_psi4_max_Lev3[2],
    't_fmax,h [R01]': t_diff_h_1,
    't_fmax,h [R02]': t_diff_h_2,
    't_fmax,h [R03]': t_diff_h_3,
    't_fmax,psi4 [R01]': t_diff_psi4_1,
    't_fmax,psi4 [R02]': t_diff_psi4_2,
    't_fmax,psi4 [R03]': t_diff_psi4_3,
    
})

print(Data_Summary_BAM.head())

147
147
       Run Name Code Identifier   M_chirp            Λ fmrg,h [R01]  \
0    2B_100_269  BAM    BAM0001  1.175222   126.726240      2.56943   
1    2H_100_269  BAM    BAM0002  1.175225  2325.898133     1.249122   
2  ALF2_100_270  BAM    BAM0003  1.175251   733.345067     1.650229   
3  ALF2_100_270  BAM    BAM0004  1.176082   730.080533     1.659674   
4  ALF2_100_275  BAM    BAM0005  1.197014   658.146133     1.695904   

  fmrg,h [R02] fmrg,h [R03] fmrg,psi4 [R01] fmrg,psi4 [R02]  ...  \
0      2.55538     2.492708        4.061753        3.917622  ...   
1     1.237124            -        1.678352        1.583489  ...   
2      1.63695            -        2.681393        2.728979  ...   
3     1.658757            -        2.773417        2.897306  ...   
4     1.680151            -        6.898144        6.719053  ...   

  A_max,h [R03] A_max,psi4 [R01] A_max,psi4 [R02] A_max,psi4 [R03]  \
0      0.011941         0.005296         0.005076         0.010629   
1             - 

It seems a bit odd that the time between merger and max frequency is different for the strain and $\Psi_4$ data. I will need to look into this.

## Save the Summary Dataframe

In [64]:
#Make path if it is not made yet
import os
folder = './Summary_Data'
os.makedirs(folder, exist_ok=True)

#Define Path
path = folder+'/V1_BAM_Summary.txt'

Data_Summary_BAM.to_csv(path, sep='\t', index=False)

In [65]:
test = pd.read_csv(path, sep='\t')
print(test.head())

       Run Name Code Identifier   M_chirp            Λ        fmrg,h [R01]  \
0    2B_100_269  BAM    BAM0001  1.175222   126.726240  2.5694300854899597   
1    2H_100_269  BAM    BAM0002  1.175225  2325.898133  1.2491221651299282   
2  ALF2_100_270  BAM    BAM0003  1.175251   733.345067   1.650229100921526   
3  ALF2_100_270  BAM    BAM0004  1.176082   730.080533  1.6596743749638427   
4  ALF2_100_275  BAM    BAM0005  1.197014   658.146133  1.6959040010450448   

         fmrg,h [R02]        fmrg,h [R03]     fmrg,psi4 [R01]  \
0   2.555380285681122  2.4927078022503966  4.0617532151463145   
1   1.237124238731809                   -  1.6783515295323224   
2   1.636949618836252                   -   2.681393337388867   
3   1.658756802479667                   -   2.773416748131237   
4  1.6801510513015794                   -   6.898143605770027   

      fmrg,psi4 [R02]  ...         A_max,h [R03]      A_max,psi4 [R01]  \
0  3.9176218372178693  ...  0.011940624187736339  0.00529581624924

## Save to Excel File

In [67]:
#Make path if it is not made yet
import os
folder = './Summary_Data'
os.makedirs(folder, exist_ok=True)

#Define Path
path = folder+'/V1_BAM_Summary.xlsx'

Data_Summary_BAM.to_excel(path, index=False)